**Finding Top Similar Words**

Author: Lucas Ma

In [7]:
# Import libraries
from transformers import AutoTokenizer, AutoModel
import torch
from torch.nn.functional import cosine_similarity
import numpy as np

# Load the tokenizer and model
# Load the fine-tuned MacBERTh model and tokenizer
model_name = './fine-tuned-MacBERTh'  # Path to your fine-tuned model
# model_name = "emanjavacas/MacBERTh"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get word embedding
def get_word_embedding(word, tokenizer, model):
    inputs = tokenizer(word, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.squeeze()

def find_top_similar_words(target_word, text, tokenizer, model, top_n=10):
    # Tokenize and get embeddings for the target word
    target_embedding = get_word_embedding(target_word, tokenizer, model)
    
    # Tokenize the input text
    words = tokenizer.tokenize(text)
    word_embeddings = []
    
    for word in words:
        embedding = get_word_embedding(word, tokenizer, model)
        word_embeddings.append(embedding)
    
    # Calculate similarities
    similarities = []
    for word, embedding in zip(words, word_embeddings):
        similarity = cosine_similarity(target_embedding.unsqueeze(0), embedding.unsqueeze(0)).item()
        similarities.append((word, similarity))
    
    # Sort similarities
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Filter out duplicates and select top N unique words
    unique_words = []
    seen_words = set()
    for word, similarity in similarities:
        if word not in seen_words:
            unique_words.append((word, similarity))
            seen_words.add(word)
        if len(unique_words) >= top_n:
            break
    
    return unique_words

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Path to the .txt file
file_path = 'data/bible_full_text.txt'

# Read the text from the file
text = read_text_from_file(file_path)

# Target word to find similarities with
target_word = "natiue"

# Find and print top 10 similar words
top_similar_words = find_top_similar_words(target_word, text, tokenizer, model, top_n=10)
print(f"Top 10 words most similar to '{target_word}':")
for word, similarity in top_similar_words:
    print(f"{word}: {similarity}")


Some weights of BertModel were not initialized from the model checkpoint at ./fine-tuned-MacBERTh and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 